In [3]:
#Read in Packages
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import numpy as np
from geopandas import GeoDataFrame
from numpy.random import RandomState, uniform
import string
from sodapy import Socrata
import seaborn as sns
import networkx as nx
from networkx.algorithms import community
import community.community_louvain as community
from datetime import datetime
from pytz import timezone
import statsmodels.api as sm
import json
%matplotlib inline


C:\Users\brandin_hirsch\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The goal of this notebook is to load in the millions of rideshare rides into a single dataframe, clean, then export this data for further analysis.

In [ ]:
#API here if needed
# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# # Example authenticated client (needed for non-public datasets):
# # client = Socrata(data.cityofchicago.org,
# #                  MyAppToken,
# #                  userame="user@example.com",
# #                  password="AFakePassword")

# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("m6dm-c72p", limit=8000000)

# # Convert to pandas DataFrame
# results_df = pd.DataFrame.from_records(results)

#Specify Columns to Use
usecols=['Trip Start Timestamp',
 'Trip End Timestamp',
 'Trip Seconds',
 'Trip Miles',
 'Pickup Census Tract',
 'Dropoff Census Tract',
 'Fare',
 'Tip',
 'Additional Charges',
 'Trip Total',
 'Trips Pooled',
 'Pickup Centroid Latitude',
 'Pickup Centroid Longitude',
 'Dropoff Centroid Latitude',
 'Dropoff Centroid Longitude']


#Read in data
results_df=pd.read_csv(r'C:\Users\brandin_hirsch\Downloads\Transportation_Network_Providers_-_Trips (1).csv',usecols=usecols,dtype={'Dropoff Census Tract':'object','Pickup Census Tract':'object'})

#Read in map of Chicago
chicago_map=gpd.read_file(r'C:\Users\brandin_hirsch\Desktop\chicago\geo_export_e91837ed-0772-4931-9304-2631f6936d17.shp')

In [8]:
results_df.tail()

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Tip,Additional Charges,Trip Total,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude
45338594,12/11/2018 11:45:00 AM,12/11/2018 12:15:00 PM,"1,996",22.801534,NaN,17031081500,30,0.0,2.5,32.5,1,NaN,NaN,41.892508,-87.626215
45338595,11/06/2018 06:00:00 PM,11/06/2018 06:30:00 PM,"1,170",3.219869,17031080300,17031833000,10,0.0,2.5,12.5,1,41.907492,-87.635760,41.885281,-87.657233
45338596,12/23/2018 09:45:00 PM,12/23/2018 10:00:00 PM,488,2.322259,17031081500,17031330100,5,0.0,2.5,7.5,1,41.892508,-87.626215,41.859350,-87.617358
45338597,11/09/2018 09:00:00 AM,11/09/2018 09:15:00 AM,912,2.428399,17031243300,17031839100,7.5,0.0,2.5,10,1,41.892493,-87.664746,41.880994,-87.632746
45338598,11/02/2018 05:30:00 PM,11/02/2018 05:30:00 PM,513,1.720073,17031320100,17031081402,5,0.0,7.5,12.5,1,41.884987,-87.620993,41.891972,-87.612945


After reading in the dataframe, cleanse data, create dataframes for network nodes and network weights for network analysis

In [9]:
#Drop trips entirely outside of Chicago
results_df=results_df.dropna(subset=['Dropoff Census Tract','Pickup Census Tract','Dropoff Centroid Latitude','Pickup Centroid Latitude'],how='any')


# results_df=results_df.dropna(subset=['dropoff_census_tract','pickup_census_tract'],how='all')

# results_df=results_df.dropna(subset=['dropoff_centroid_latitude','pickup_centroid_latitude'],how='all')

results_df['dropoff_census_tract']=results_df['Dropoff Census Tract']

results_df['pickup_census_tract']=results_df['Pickup Census Tract']

results_df['dropoff_centroid_latitude']=results_df['Dropoff Centroid Latitude']

results_df['pickup_centroid_latitude']=results_df['Pickup Centroid Latitude']

results_df['trip_count']=1

 
#Calculating Network Values

#Create network analysis df to make sure every census tract we plot has a lat/long
network_analysis_df=results_df.dropna(subset=['dropoff_census_tract','pickup_census_tract','dropoff_centroid_latitude','pickup_centroid_latitude'],how='any')

#Create new empty dataframe for network analysis values
df_network_values=pd.DataFrame()

#Take the columns for start and end location and trip count into the network weights dataframe
df_network_values['start_location']=network_analysis_df['pickup_census_tract']

df_network_values['end_location']=network_analysis_df['dropoff_census_tract']

df_network_values['trip_count']=1

df_network_values=df_network_values.groupby(['start_location','end_location'])['trip_count'].sum()

df_network_values=pd.DataFrame(df_network_values)

df_network_values.reset_index(inplace=True)


#Calculating Network Weights

#Calculate sum of pickups and sum of dropoffs at each location 
pickups_sum=df_network_values[['start_location','trip_count']]

dropoffs_sum=df_network_values[['end_location','trip_count']]

#Rename columns for easy joining
pickups_sum.columns=['census_tract','trip_count']
dropoffs_sum.columns=['census_tract','trip_count']

#Join columns to get total pickups and dropoffs in each location
df_network_weights=pd.concat([pickups_sum,dropoffs_sum])

df_network_weights=pd.DataFrame(df_network_weights.groupby('census_tract')['trip_count'].sum())

df_network_weights.reset_index(inplace=True)

df_network_weights.sort_values(by='trip_count',ascending=False,inplace=True)



len(df_network_weights.census_tract.unique())

802

In [45]:
network_analysis_df.head()

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Fare,Tip,Additional Charges,Trip Total,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude,dropoff_census_tract,pickup_census_tract,dropoff_centroid_latitude,pickup_centroid_latitude,trip_count
0,11/17/2018 05:00:00 PM,11/17/2018 05:15:00 PM,578,1.821470,17031080202,17031070101,5,0.0,2.5,7.5,1,41.905858,-87.630865,41.929263,-87.635891,17031070101,17031080202,41.929263,41.905858,1
1,11/10/2018 05:15:00 PM,11/10/2018 05:45:00 PM,"1,163",3.903087,17031241400,17031081201,10,0.0,2.5,12.5,1,41.906026,-87.675312,41.899156,-87.626211,17031081201,17031241400,41.899156,41.906026,1
2,12/07/2018 07:15:00 PM,12/07/2018 07:30:00 PM,766,2.072686,17031071300,17031081700,7.5,1.0,2.5,11,1,41.921778,-87.641460,41.892042,-87.631864,17031081700,17031071300,41.892042,41.921778,1
3,12/30/2018 10:00:00 PM,12/30/2018 10:30:00 PM,"1,543",4.948383,17031240300,17031081600,5,0.0,2.5,7.5,3,41.912364,-87.675063,41.892073,-87.628874,17031081600,17031240300,41.892073,41.912364,1
4,12/20/2018 11:00:00 PM,12/20/2018 11:15:00 PM,473,1.930627,17031832200,17031242000,5,0.0,2.5,7.5,1,41.920452,-87.679955,41.899737,-87.664954,17031242000,17031832200,41.899737,41.920452,1


Create dataframe for all time series across all census tracts for forecasting



In [17]:
#Create new empty dataframe for grouping every row by time, pickup, dropoff location
df_trips_all_locations=pd.DataFrame()

#Take the necessary columns from network_analysis_df

df_trips_all_locations['start_location']=network_analysis_df['pickup_census_tract']

df_trips_all_locations['end_location']=network_analysis_df['dropoff_census_tract']

df_trips_all_locations['start_time']=network_analysis_df['Trip Start Timestamp']

df_trips_all_locations['end_time']=network_analysis_df['Trip End Timestamp']

df_trips_all_locations['trip_count']=1

#Group by time, pickup, dropoff location
df_trips_all_locations=df_trips_all_locations.groupby(['start_location','end_location','start_time','end_time'])['trip_count'].sum()

df_trips_all_locations=pd.DataFrame(df_trips_all_locations)

df_trips_all_locations.reset_index(inplace=True)

#Calculate sum of pickups and sum of dropoffs at each location 
df_trips_all_pickups=df_trips_all_locations[['start_time','start_location','trip_count']]

df_trips_all_dropoffs=df_trips_all_locations[['end_time','end_location','trip_count']]

#Rename columns for easy joining
df_trips_all_pickups.columns=['time','census_tract','trip_count']
df_trips_all_dropoffs.columns=['time','census_tract','trip_count']

#Join columns to get total pickups and dropoffs in each location
df_trips_all_locations=pd.concat([df_trips_all_pickups,df_trips_all_dropoffs])

#Create dataframe for all trips in all locations from Nov 2018-March 2019
df_trips_all_locations=pd.DataFrame(df_trips_all_locations.groupby(['time','census_tract'])['trip_count'].sum())

df_trips_all_locations.reset_index(inplace=True)

df_trips_all_locations.sort_values(by='time',ascending=False,inplace=True)

df_trips_all_locations.head()

,time,census_tract,trip_count
0,01/01/2019 04:15:00 AM,17031010100,1
1,01/01/2019 10:45:00 PM,17031010100,1
2,01/02/2019 01:30:00 PM,17031010100,1
3,01/04/2019 11:15:00 AM,17031010100,1
4,01/05/2019 09:45:00 PM,17031010100,1


Create list containing all census tracts and lat long coordinates for plotting network nodes

In [ ]:
#Create list of all combinations of census tract links with latitude and longitude as locations to plot the network map

pickup_census=network_analysis_df[['pickup_census_tract','pickup_centroid_latitude','Pickup Centroid Longitude']]
dropoff_census=network_analysis_df[['dropoff_census_tract','dropoff_centroid_latitude','Dropoff Centroid Longitude']]

#Rename columns for easy joining
columns=['census_tract','Lat','Long']

pickup_census.columns=columns
dropoff_census.columns=columns

pos_data=pd.concat([pickup_census,dropoff_census])

pos_data.drop_duplicates(subset=['census_tract'],inplace=True)

pos_data.Long=pos_data.Long.astype(float)
pos_data.Lat=pos_data.Lat.astype(float)



Export All Finished Dataframes

In [162]:
#Export all cleansed and finished dataframes

#File of all time series for each census tract
df_trips_all_locations.to_csv(r'C:\Users\brandin_hirsch\Desktop\full_data_nov_march.csv')

#Values used to create the nodes and edges in the network
df_network_values.to_csv(r'C:\Users\brandin_hirsch\Desktop\network_node_values_nov_march.csv')

#Values used as network node weights
df_network_weights.to_csv(r'C:\Users\brandin_hirsch\Desktop\network_weights_nov_march.csv')

#Position data for each census tract

pos_data.to_csv(r'C:\Users\brandin_hirsch\Desktop\pos_data_v3.csv')
